# New Zealand DEM remake

See the original notebook [here](https://nbviewer.org/github/royalosyin/Work-with-DEM-data-using-Python-from-Simple-to-Complicated/blob/master/Sup03-Ridgelines%20Map%20of%20DEM.ipynb).

In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
def dataset_array_to_dataframe(dataset_array):
    df = pd.DataFrame.from_records([
        (j, i, a)
        for i, r in enumerate(dataset_array)
        for j, a in enumerate(r)
    ], columns=["x", "y", "h"])
    return df

def process_rows(df, step=1):
    def add_tails_to_row(subdf, y):
        xmin, xmax = subdf['x'].min(), subdf['x'].max()
        return pd.concat([
            subdf,
            pd.DataFrame({'x': [xmin - 1, xmax + 1], 'y': [y, y], 'h': [0, 0]})
        ], ignore_index=True).assign(h_max=lambda d: d['h'].max())
    return pd.concat([
        add_tails_to_row(df[df['y'] == y], y) for y in range(df['y'].min(), df['y'].max() + 1, step)
    ]).sort_values(by=['y', 'x']).reset_index(drop=True)

In [4]:
raw_data_array = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/new_zealand.csv", header=None).to_numpy()
df = dataset_array_to_dataframe(raw_data_array)
df = process_rows(df[df["h"] > 0], 2)
bbox = dict(xmin=df['x'].min(), ymin=df['y'].min(), xmax=df['x'].max(), ymax=df['y'].max())
print(df.shape)
df.head()

(4243, 4)


,x,y,h,h_max
0,432,6,0.000000,180.143860
1,433,6,114.518776,180.143860
2,435,6,180.143860,180.143860
3,436,6,0.000000,180.143860
4,420,8,0.000000,124.377769


In [5]:
ggplot(df) + \
    geom_area_ridges(aes("x", "y", height="h"), \
                     stat='identity', scale=.005, \
                     color="#08519c", fill="#bdd7e7", \
                     sampling=sampling_pick(df.shape[0]), \
                     tooltips=layer_tooltips().line("height|@h").format("@h", 'd'), \
                     show_legend=False) + \
    geom_text(x=bbox['xmin'] + .7 * (bbox['xmax'] - bbox['xmin']), \
              y=bbox['ymin'] + .9 * (bbox['ymax'] - bbox['ymin']), \
              label="New Zealand", size=25, family="Cinzel") + \
    scale_y_continuous(trans='reverse') + \
    ggsize(600, 600) + \
    theme_minimal() + \
    theme(axis='blank', panel_grid='blank', \
          plot_background=element_rect(color='black', fill='#e6e6e6', size=1))